Dataset: "SLU_Opportunity_Wise_Data" (csv file) [Raw Dataset]

Importing Necessary Libraries

In [17]:
import pandas as pd
import numpy as np
import re
import missingno as msno
from sklearn.impute import SimpleImputer

Reading the CSV file

In [18]:
file_path = "SLU_Opportunity_Wise_Data.csv"  
df = pd.read_csv(file_path)

Understanding Dataset Information

In [19]:
# Basic info about dataset (columns, dtypes, null counts, memory)
df.info()

# Statistical summary (numeric columns)
df.describe()

# Quick look at first rows
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8558 entries, 0 to 8557
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Learner SignUp DateTime  8558 non-null   object
 1   Opportunity Id           8558 non-null   object
 2   Opportunity Name         8558 non-null   object
 3   Opportunity Category     8558 non-null   object
 4   Opportunity End Date     8558 non-null   object
 5   First Name               8558 non-null   object
 6   Date of Birth            8558 non-null   object
 7   Gender                   8558 non-null   object
 8   Country                  8558 non-null   object
 9   Institution Name         8553 non-null   object
 10  Current/Intended Major   8553 non-null   object
 11  Entry created at         8558 non-null   object
 12  Status Description       8558 non-null   object
 13  Status Code              8558 non-null   int64 
 14  Apply Date               8558 non-null  

,Learner SignUp DateTime,Opportunity Id,Opportunity Name,Opportunity Category,Opportunity End Date,First Name,Date of Birth,Gender,Country,Institution Name,Current/Intended Major,Entry created at,Status Description,Status Code,Apply Date,Opportunity Start Date
0,06/14/2023 12:30:35,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,06/29/2024 18:52:39,Faria,01/12/2001,Female,Pakistan,Nwihs,Radiology,03/11/2024 12:01:41,Started,1080,06/14/2023 12:36:09,11/03/2022 18:30:39
1,05/01/2023 05:29:16,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,06/29/2024 18:52:39,Poojitha,08/16/2000,Female,India,SAINT LOUIS,Information Systems,03/11/2024 12:01:41,Started,1080,05/01/2023 06:08:21,11/03/2022 18:30:39
2,04/09/2023 20:35:08,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,06/29/2024 18:52:39,Emmanuel,01/27/2002,Male,United States,Illinois Institute of Technology,Computer Science,03/11/2024 12:01:41,Started,1080,05/11/2023 1085640:21:29,11/03/2022 18:30:39
3,08/29/2023 05:20:03,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,06/29/2024 18:52:39,Amrutha Varshini,11/01/1999,Female,United States,Saint Louis University,Information Systems,03/11/2024 12:01:41,Team Allocated,1070,10/09/2023 22:02:42,11/03/2022 18:30:39
4,01/06/2023 15:26:36,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,06/29/2024 18:52:39,Vinay Varshith,04/19/2000,Male,United States,Saint Louis University,Computer Science,03/11/2024 12:01:41,Started,1080,01/06/2023 15:40:10,11/03/2022 18:30:39


1. Handling Missing & NULL Values

 Checking total and percentage of missing values

In [20]:
missing_counts = df.isnull().sum()
missing_percentage = (df.isnull().mean() * 100).round(2)

print("Missing Value Counts:\n", missing_counts)
print("\nMissing Value Percentage:\n", missing_percentage)

Missing Value Counts:
 Learner SignUp DateTime       0
Opportunity Id                0
Opportunity Name              0
Opportunity Category          0
Opportunity End Date          0
First Name                    0
Date of Birth                 0
Gender                        0
Country                       0
Institution Name              5
Current/Intended Major        5
Entry created at              0
Status Description            0
Status Code                   0
Apply Date                    0
Opportunity Start Date     3794
dtype: int64

Missing Value Percentage:
 Learner SignUp DateTime     0.00
Opportunity Id              0.00
Opportunity Name            0.00
Opportunity Category        0.00
Opportunity End Date        0.00
First Name                  0.00
Date of Birth               0.00
Gender                      0.00
Country                     0.00
Institution Name            0.06
Current/Intended Major      0.06
Entry created at            0.00
Status Description          

Handled Missing Values

In [21]:
# Fill missing values for the two categorical columns
df[["Institution Name", "Current/Intended Major"]] = df[["Institution Name", "Current/Intended Major"]].fillna("None")

# Handle Opportunity Start Date
date_cols = ["Opportunity Start Date"]
from sklearn.impute import SimpleImputer
date_imputer = SimpleImputer(strategy="most_frequent")  # quick solution
df[date_cols] = date_imputer.fit_transform(df[date_cols])

Verifying remaining missing values

In [22]:
print("Remaining Missing Values:\n", df[["Institution Name", "Current/Intended Major", "Opportunity Start Date"]].isnull().sum())

Remaining Missing Values:
 Institution Name          0
Current/Intended Major    0
Opportunity Start Date    0
dtype: int64


Note: For categorical columns such as Institution Name and Current/Intended Major, only a very small number of missing values were present, which were safely filled with "None". And in Date columns such as Opportunity Start Date, large number of missing values were present, which were safely filled with most frequent date values. Otherwise All other categorical columns were complete and required no changes.

2. Handling Outliers

Detect Outliers Using IQR (Safe Method)

In [23]:
# Numeric column
num_col = "Status Code"

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df[num_col].quantile(0.25)
Q3 = df[num_col].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Flag outliers
outliers = df[(df[num_col] < lower_bound) | (df[num_col] > upper_bound)]
print(f"Number of outliers in {num_col}: {len(outliers)}")

# Optional: view outlier rows
print(outliers)


Number of outliers in Status Code: 0
Empty DataFrame
Columns: [Learner SignUp DateTime, Opportunity Id, Opportunity Name, Opportunity Category, Opportunity End Date, First Name, Date of Birth, Gender, Country, Institution Name, Current/Intended Major, Entry created at, Status Description, Status Code, Apply Date, Opportunity Start Date]
Index: []


Note: We analyzed the dataset for outliers to ensure data quality. For the numeric column Status Code, the Interquartile Range (IQR) method revealed no significant deviations, indicating that all values are within a reasonable range. Since categorical columns do not have outliers in the traditional sense, no action was needed for them. Overall, the dataset is clean, with no extreme values present, and no modifications were required that could introduce missing values or distort the data.

3. Standardizing formats

Standardize of Categorical Columns

In [24]:
# Strip extra spaces and convert to title case
categorical_cols = ["Gender", "Country", "Institution Name", "Current/Intended Major"]

for col in categorical_cols:
    df[col] = df[col].astype(str).str.strip().str.title()

Note: The categorical columns, including Gender, Country, Institution Name, and Current/Intended Major, were cleaned by removing any extra spaces and standardizing the text to title case. This ensures consistency across the dataset, reduces errors caused by variations in spelling or capitalization, and facilitates accurate analysis, grouping, and visualization of categorical data.

Standardize of Date Columns

In [25]:
# List of date columns
date_cols = ['Learner SignUp DateTime', 'Opportunity Start Date', 'Apply Date', 
             'Opportunity End Date', 'Entry created at']

# Check current formats
for col in date_cols:
    print(f"Column: {col}")
    print(df[col].head())
    print(df[col].dtype)
    print("-"*50)

Column: Learner SignUp DateTime
0    06/14/2023 12:30:35
1    05/01/2023 05:29:16
2    04/09/2023 20:35:08
3    08/29/2023 05:20:03
4    01/06/2023 15:26:36
Name: Learner SignUp DateTime, dtype: object
object
--------------------------------------------------
Column: Opportunity Start Date
0    11/03/2022 18:30:39
1    11/03/2022 18:30:39
2    11/03/2022 18:30:39
3    11/03/2022 18:30:39
4    11/03/2022 18:30:39
Name: Opportunity Start Date, dtype: object
object
--------------------------------------------------
Column: Apply Date
0         06/14/2023 12:36:09
1         05/01/2023 06:08:21
2    05/11/2023 1085640:21:29
3         10/09/2023 22:02:42
4         01/06/2023 15:40:10
Name: Apply Date, dtype: object
object
--------------------------------------------------
Column: Opportunity End Date
0    06/29/2024 18:52:39
1    06/29/2024 18:52:39
2    06/29/2024 18:52:39
3    06/29/2024 18:52:39
4    06/29/2024 18:52:39
Name: Opportunity End Date, dtype: object
object
--------------------

Correcting the formats of Opportunity End Date, Opportunity Start Date & Learner SignUp DateTime

In [26]:
def fix_corrupted_datetime(x):
    if pd.isna(x):
        return pd.Timestamp("1900-01-01 00:00")  # placeholder if completely empty
    x = str(x).strip()
    # Extract valid date and time digits only
    # Keep only first 10 chars for date and first 8 chars for time after space
    if " " in x:
        date_part, time_part = x.split(" ", 1)
        date_part = re.sub(r"[^\d/-]", "", date_part)[:10]  # keep only digits and / or -
        time_part = re.sub(r"[^\d:]", "", time_part)[:8]     # keep only digits and :
        fixed_str = f"{date_part} {time_part}"
    else:
        fixed_str = re.sub(r"[^\d/-]", "", x)[:10] + " 00:00:00"  # add default time
    # Try parsing
    try:
        return pd.to_datetime(fixed_str, errors='coerce', dayfirst=False)
    except:
        return pd.Timestamp("1900-01-01 00:00")

# Apply to Opportunity End Date
df['Opportunity End Date'] = df['Opportunity End Date'].apply(fix_corrupted_datetime)

# Optional: uniform display format
df['Opportunity End Date'] = df['Opportunity End Date'].dt.strftime("%m/%d/%Y %H:%M:%S")

print(df['Opportunity End Date'].head())

0    06/29/2024 18:52:39
1    06/29/2024 18:52:39
2    06/29/2024 18:52:39
3    06/29/2024 18:52:39
4    06/29/2024 18:52:39
Name: Opportunity End Date, dtype: object


In [27]:
def fix_opportunity_start(x):
    if pd.isna(x):
        return pd.Timestamp("1900-01-01 00:00")  # placeholder for empty
    x_str = str(x).strip()

    # Case 1: If it's a number (Excel serial date)
    if x_str.isdigit():
        try:
            return pd.to_datetime(int(x_str), origin='1899-12-30', unit='D')  # Excel date origin
        except:
            return pd.Timestamp("1900-01-01 00:00")
    
    # Case 2: Corrupted datetime string
    if " " in x_str:
        date_part, time_part = x_str.split(" ", 1)
        date_part = re.sub(r"[^\d/-]", "", date_part)[:10]  # keep only date digits
        time_part = re.sub(r"[^\d:]", "", time_part)[:8]    # keep only time digits
        fixed_str = f"{date_part} {time_part}"
    else:
        fixed_str = re.sub(r"[^\d/-]", "", x_str)[:10] + " 00:00:00"
    
    # Parse the cleaned string
    try:
        return pd.to_datetime(fixed_str, errors='coerce', dayfirst=False)
    except:
        return pd.Timestamp("1900-01-01 00:00")

# Apply to Opportunity Start Date
df['Opportunity Start Date'] = df['Opportunity Start Date'].apply(fix_opportunity_start)

# Optional: uniform format
df['Opportunity Start Date'] = df['Opportunity Start Date'].dt.strftime("%m/%d/%Y %H:%M:%S")

print(df['Opportunity Start Date'].head())


0    11/03/2022 18:30:39
1    11/03/2022 18:30:39
2    11/03/2022 18:30:39
3    11/03/2022 18:30:39
4    11/03/2022 18:30:39
Name: Opportunity Start Date, dtype: object


In [28]:
def fix_learner_signup(x):
    if pd.isna(x):
        return pd.Timestamp("1900-01-01 00:00")  # placeholder for empty
    x_str = str(x).strip()

    # Case 1: If it's a number (Excel serial date, including decimals)
    try:
        val = float(x_str.split()[0])  # take first part if extra text exists
        return pd.to_datetime(val, origin='1899-12-30', unit='D')
    except:
        pass

    # Case 2: Corrupted datetime string
    if " " in x_str:
        date_part, time_part = x_str.split(" ", 1)
        date_part = re.sub(r"[^\d/-]", "", date_part)[:10]  # keep only date digits
        time_part = re.sub(r"[^\d:]", "", time_part)[:8]    # keep only time digits
        fixed_str = f"{date_part} {time_part}"
    else:
        fixed_str = re.sub(r"[^\d/-]", "", x_str)[:10] + " 00:00:00"

    # Parse the cleaned string
    try:
        return pd.to_datetime(fixed_str, errors='coerce', dayfirst=False)
    except:
        return pd.Timestamp("1900-01-01 00:00")

# Apply to Learner SignUp DateTime
df['Learner SignUp DateTime'] = df['Learner SignUp DateTime'].apply(fix_learner_signup)

# Optional: uniform display format
df['Learner SignUp DateTime'] = df['Learner SignUp DateTime'].dt.strftime("%m/%d/%Y %H:%M:%S")

print(df['Learner SignUp DateTime'].head())

0    06/14/2023 12:30:35
1    05/01/2023 05:29:16
2    04/09/2023 20:35:08
3    08/29/2023 05:20:03
4    01/06/2023 15:26:36
Name: Learner SignUp DateTime, dtype: object


In [47]:
def fix_apply_date(x):
    if pd.isna(x):
        return pd.Timestamp("1900-01-01 00:00")  # placeholder for empty
    x_str = str(x).strip()

    # Case 1: Excel serial number (integer or float like 45316.04009)
    if x_str.replace(".", "", 1).isdigit():
        try:
            return pd.to_datetime(float(x_str), origin='1899-12-30', unit='D')
        except:
            return pd.Timestamp("1900-01-01 00:00")

    # Case 2: Corrupted datetime string like "05/11/2023 1085640:21:29"
    if " " in x_str:
        parts = x_str.split(" ", 1)
        date_part = re.sub(r"[^\d/-]", "", parts[0])[:10]  # keep digits, /, -
        time_part = re.sub(r"[^\d:]", "", parts[1])[:8] if len(parts) > 1 else "00:00:00"
        fixed_str = f"{date_part} {time_part}"
    else:
        fixed_str = re.sub(r"[^\d/-]", "", x_str)[:10] + " 00:00:00"

    # Try parsing
    parsed = pd.to_datetime(fixed_str, errors='coerce', dayfirst=False)
    if pd.isna(parsed):  # if parsing failed, return safe default
        return pd.Timestamp("1900-01-01 00:00")
    return parsed

# Apply to Apply Date
df['Apply Date'] = df['Apply Date'].apply(fix_apply_date)

# Uniform format (MM/DD/YYYY HH:MM:SS AM/PM)
df['Apply Date'] = df['Apply Date'].dt.strftime("%m/%d/%Y %I:%M:%S %p")

print(df['Apply Date'].head())

0    06/14/2023 12:36:09 PM
1    05/01/2023 06:08:21 AM
2    01/01/1900 12:00:00 PM
3    10/09/2023 10:02:42 AM
4    01/06/2023 03:40:10 AM
Name: Apply Date, dtype: object


Note: The date columns in the dataset, only Entry Created At, were already in a consistent datetime64 format. This ensures uniformity across all records, making date-based analysis and comparisons straightforward.The Columns - Learner SignUp DateTime, Opportunity Start Date, Opportunity End Date and Apply Date had to changed the format because they had these type of format issue "04/12/2024 1095120:00:00","06/29/2024 18:52:39" etc. So converted these into corrected format.

Checking Numeric Columns if needs Standardize

In [30]:
# Numeric columns
num_cols = ['Status Code']

for col in num_cols:
    print(f"Column: {col}")
    print(df[col].describe())
    print(df[col].dtype)
    print("-"*50)

Column: Status Code
count    8558.000000
mean     1052.225987
std        21.665207
min      1010.000000
25%      1030.000000
50%      1050.000000
75%      1070.000000
max      1120.000000
Name: Status Code, dtype: float64
int64
--------------------------------------------------


Note: The Status Code column in the dataset is already clean and consistent. All values are of integer type (int64) and fall within a reasonable range, with no extreme outliers or irregularities. As a result, no further standardization is required, and the column is ready for analysis or any downstream processing.

4. Correcting Errors

Correcting short forms into full forms of Institution name column's Values based on country. Also correcting country column's values short form into full forms.

In [32]:
# ------------------ STEP 1 Clean column names ------------------
df.columns = df.columns.str.strip()

# ------------------ STEP 2 Standardize Country Names ------------------
country_mapping = {
    'US': 'United States', 'USA': 'United States', 'U.S.': 'United States',
    'UK': 'United Kingdom', 'IND': 'India', 'PK': 'Pakistan', 'BD': 'Bangladesh',
    'GH': 'Ghana', 'NG': 'Nigeria', 'ET': 'Ethiopia', 'RW': 'Rwanda',
    'KE': 'Kenya', 'ZA': 'South Africa', 'KR': 'South Korea', 'PH': 'Philippines',
    'ZM': 'Zambia', 'ES': 'Spain', 'EG': 'Egypt', 'YE': 'Yemen'
}

df['Country'] = df['Country'].replace(country_mapping)

valid_countries = [
    'United States', 'India', 'Nigeria', 'Ghana', 'Ethiopia', 'Rwanda', 'Kenya',
    'South Africa', 'Pakistan', 'Bangladesh', 'Philippines', 'Zambia', 'Spain',
    'Egypt', 'Yemen', 'South Korea', 'United Kingdom'
]


# ------------------ STEP 3: Define full institution corrections dictionary ------------------
institution_corrections = {
    # Afghanistan
    "Gndu": "Guru Nanak Dev University",
    
    # Azerbaijan
    "Ashoka": "Ashoka University",
    
    # Bangladesh
    "Aust": "Ahsanullah University of Science and Technology",
    "Cpscr": "Chattogram Port School and College",

    # British Indian Ocean Territory
    "Asdads": "Unknown Institution",

    # China
    "长沙学院": "Changsha University",

    # Egypt
    "Bue": "The British University in Egypt",
    "Must": "Misr University for Science and Technology",
    "Habiba": "Habiba Community School",

    # Ghana
    "Upsa": "University of Professional Studies, Accra",
    "Knust": "Kwame Nkrumah University of Science and Technology",
    "Aamusted": "Akenten Appiah-Menka University of Skills Training and Entrepreneurial Development",
    # India
    "Jntuh": "Jawaharlal Nehru Technological University Hyderabad",
    "Gitam": "Gandhi Institute of Technology and Management",
    "Nift": "National Institute of Fashion Technology",
    "Ignou": "Indira Gandhi National Open University",
    "Lpu": "Lovely Professional University",
    "Vtu": "Visvesvaraya Technological University",
    "Nituk": "National Institute of Technology Uttarakhand",
    "Psit": "Pranveer Singh Institute of Technology",
    "Srm": "SRM Institute of Science and Technology",
    "Nit": "National Institute of Technology",
    "Msu": "Maharaja Sayajirao University of Baroda",
    "IIT Delhi": "Indian Institute of Technology Delhi",
    "IIT Ism Dhanbad": "Indian Institute of Technology (ISM) Dhanbad",
    "NIT Durgapur": "National Institute of Technology Durgapur",
    "Nit-Agartala": "National Institute of Technology Agartala",
    "Bits Pilani Hyderabad Campus": "Birla Institute of Technology and Science, Pilani - Hyderabad Campus",
    "VIT University": "Vellore Institute of Technology",
    "Vit Ap University": "Vellore Institute of Technology - Andhra Pradesh",
    "SRM University": "SRM Institute of Science and Technology",
    "JNTU Kakinada": "Jawaharlal Nehru Technological University Kakinada",
    "Jntuhceh": "Jawaharlal Nehru Technological University Hyderabad",
    "IIIT Rk Valley": "Indian Institute of Information Technology RK Valley",
    "IIM Kozhikode": "Indian Institute of Management Kozhikode",
    "IIM Nagpur": "Indian Institute of Management Nagpur",
    "Jntuacek": "Jawaharlal Nehru Technological University Anantapur",
    "Jntuk": "Jawaharlal Nehru Technological University Kakinada",
    "M. G University": "Mahatma Gandhi University",
    "Mjcet": "Muffakham Jah College of Engineering and Technology",
    "Gndit": "Guru Nanak Dev Institute of Technology",
    "Sju": "St. Joseph’s University",
    "Sies": "SIES College of Arts, Science & Commerce",
    "Sscbs": "Shaheed Sukhdev College of Business Studies",
    "Srcc": "Shri Ram College of Commerce",
    "Ramjas": "Ramjas College, University of Delhi",
    "Csjmu": "Chhatrapati Shahu Ji Maharaj University",
    "Rtmnu": "Rashtrasant Tukadoji Maharaj Nagpur University",
    "Hnbgu": "Hemvati Nandan Bahuguna Garhwal University",
    "Excelerate": "Excelerate Institute",
    "Iter": "Institute of Technical Education and Research",
    "Rvr&Jc": "R.V.R & J.C. College of Engineering",
    "Vnrvjiet": "VNR Vignana Jyothi Institute of Engineering & Technology",
    "Dps Ruby Park": "Delhi Public School Ruby Park",
    "Oakridge": "Oakridge International School",
    "Vibgyor": "Vibgyor Group of Schools",
    "Vibgyor High": "Vibgyor Group of Schools",
    "Jiet": "Jodhpur Institute of Engineering and Technology",
    "Lnit Srinagar": "National Institute of Technology Srinagar",
    "Tjit": "Thakur College of Engineering and Technology",
    "Cmr": "CMR Institute of Technology",
    "Cmr Technical Campus": "CMR Technical Campus",
    "Miritm": "Maturi Institute of Technology and Management",
    "Maac": "Maya Academy of Advanced Cinematics",
    "Ki University": "Kalinga Institute of Industrial Technology University",
    "K L University": "KL University",
    "Mit-Wpu": "MIT World Peace University",
    "Mit Wpu": "MIT World Peace University",
    "Mit - Wpu": "MIT World Peace University",
    "Mgit": "Mahatma Gandhi Institute of Technology",
    "Svit Vasad": "Sardar Vallabhbhai Patel Institute of Technology, Vasad",
    "Ljims": "L J Institute of Management Studies",
    "Fiem": "Future Institute of Engineering and Management",
    "Icfai University": "ICFAI University",
    "Vpkbiet": "Vidya Pratishthan’s Kamalnayan Bajaj Institute of Engineering and Technology",
    "Isb&M Pune": "International School of Business & Media, Pune",
    "Ips College": "IPS College of Technology & Management",
    "Ips Dehradun": "Institute of Professional Studies, Dehradun",
    "R V C E": "Rashtreeya Vidyalaya College of Engineering",
    "Svsps": "Sri Venkateswara Swamy Polytechnic",
    "Svnit": "Sardar Vallabhbhai National Institute of Technology",
    "Aec": "Aditya Engineering College",
    "Ihrd": "Institute of Human Resources Development",
    "Itm Sls University": "ITM SLS Baroda University",
    "Kr Mangalam": "K.R. Mangalam University",
    "Vignan": "Vignan’s Foundation for Science, Technology & Research",
    "Dseu": "Delhi Skill and Entrepreneurship University",
    "Pccoer": "Pimpri Chinchwad College of Engineering and Research",
    "P. V. G. Nashik": "Pune Vidyarthi Griha’s College of Engineering and Technology, Nashik",
    "P.V.G Nashik": "Pune Vidyarthi Griha’s College of Engineering and Technology, Nashik",
    "Biet": "Bapuji Institute of Engineering and Technology",
    "Sdes": "School of Design and Engineering Studies",
    "Ssit": "Sri Siddhartha Institute of Technology",
    "Mvj College of Engineering": "MVJ College of Engineering",
    "Gec Rajkot": "Government Engineering College Rajkot",
    "Dvr &D.Hs Mic College Of Technology": "DVR & DHS MIC College of Technology",
    "Upgrad": "UpGrad Learning Institute",
    "Bennet": "Bennett University",
    "Helpline": "Invalid University",
    "Afsm": "Invalid University",
    "Outr": "Invalid University",
    "Cdwcnwj": "Invalid University",
    "AsdfVvit": "Invalid University",
    "Asdf": "Invalid University",
    "Abc": "Invalid University",
    "Nil": "Invalid University",
    "Test": "Invalid University",
    "Ind": "Invalid University",
    "Sn": "Invalid University",
    "I": "Invalid University",
    "M.Tech": "Invalid University",
    "Popcorntime.Telugu@Gmail.Com": "Invalid University",
    "2023 Tomtom Openstreetmap Sri Venkateswara College Of Engineering" : "Sri Venkateswara College Of Engineering",
    "K L Deemed To Be University": "KL University",
    "Rmkcet": "RMK College of Engineering and Technology",
    "Codebasics": "Not Applicable",
    "Eiilm": "Eastern Institute for Integrated Learning in Management",
    "Srm Ist": "SRM Institute of Science and Technology",
    "Jntu Kakinada": "Jawaharlal Nehru Technological University Kakinada",
    "Vvit": "Vasireddy Venkatadri Institute of Technology",
    "KI University": "KL University",
    "Iim Nagpur": "Indian Institute of Management Nagpur",
    "Iiit Rk Valley": "Rajiv Gandhi University of Knowledge Technologies",
    "Mit": "Madras Institute of Technology",
    "Hindustan": "Hindustan University",
    "Aitr": "Academia-Industry Training (AIT) India program",
    "Nit Durgapur": "National Institute of Technology Durgapur",
    "Hsc": "Not applicable",
    "Isbm&Coe": "International School of Business & Management",
    "Excelerate": "Not Applicable",
    "Nxtwave": "NxtWave Institute of Advanced Technologies",
    "Iim Kozhikode": "Indian Institute of Management Kozhikode",
    "Vnit": "Visvesvaraya National Institute of Technology Nagpur",
    "Giatm": "Gandhi Institute of Technology and Management",
    "Vit Chennai": "Vellore Institute of Technology",
    "Ill": "Illinois Institute of Technology",
    "Sce Supaul": "Supaul College of Engineering",
    "Mlritm": "Marri Laxman Reddy Institute of Technology and Management",
    "Juet": "Jaypee University of Engineering and Technology", 
    "Nitesh": "Nitesh Institute Of Technology Foundation",
    "Tss": "TSS International School",
    "Sri Indu": "Sri Indu Institute of Engineering & Technology",
    "Kluniversity": "KL University",


    # Iran
    "Gds": "Graduate School of Decision Sciences",

    # Kenya
    "Jkuat": "Jomo Kenyatta University of Agriculture and Technology",

    # Lebanon
    "Lau": "Lebanese American University",

    # Malaysia
    "Utem": "Universiti Teknikal Malaysia Melaka",

    # Nigeria
    "Lasu": "Lagos State University",
    "Oauthc": "Obafemi Awolowo University Teaching Hospitals Complex",
    "Niit": "National Institute of Information Technology",
    "Fuoye": "Federal University Oye-Ekiti",
    "Uniben": "University of Benin",
    "Aa" : "None",
    "Lautech": "Ladoke Akintola University of Technology",

    # Pakistan
    "Nwihs": "Northwest Institute of Health Sciences",
    "Nust": "National University of Sciences and Technology",
    "Fuuast": "Federal Urdu University of Arts, Science and Technology",
    "Ned": "NED University of Engineering and Technology",
    "Muet": "Mehran University of Engineering and Technology",
    "Iub": "Islamia University of Bahawalpur",
    "Iobm": "Institute of Business Management",
    "Uit": "Usman Institute of Technology",
    "Lcwu": "Lahore College for Women University",
    "Gcuf": "Government College University Faisalabad",
    "Pmas": "Pir Mehr Ali Shah Arid Agriculture University",
    "Comsats": "COMSATS University Islamabad",
    "Superior": "Not Applicable",

    # Philippines
    "Joshua": "Joshua Christian Academy",

    # Rwanda
    "Unilak": "University of Lay Adventists of Kigali",
    "Nega": "New Generation Academy",
    "I Am Currently In My Gap Year": "Not Applicable",

    # South Africa
    "Umuzi": "Umuzi Academy",
    "Applying": "Not Applicable",
    "Babcock": "Babcock University",

    # UAE
    "Excelr": "ExcelR Training Institute",

    # United States
    "Pgcc": "Prince George's Community College",
    "Slu": "Saint Louis University",
    "Jntu": "Jawaharlal Nehru Technological University",
    "Qq": "Unknown Institution",
    "Purdue": "Purdue University",
    "St. Louis University": "Saint Louis University",
    "St Louis University": "Saint Louis University",
    "St Louis": "Saint Louis University",
    "Webster": "Webster University",
    "Student": "Not Applicable",
    "Employment": "Not Applicable",
    "Cu Boulder": "University of Colorado Boulder",
    "2023 Tomtom Openstreetmap Sri Venkateswara College Of Engineering": "Sri Venkateswara College Of Engineering",
}

# ------------------ STEP 4: Replace institution short forms with full names ------------------
df['Institution Name'] = df['Institution Name'].replace(institution_corrections)

# Optional: replace any remaining invalid or unmatched names with None
df['Institution Name'] = df['Institution Name'].apply(lambda x: x if isinstance(x, str) else None)

# ------------------ STEP 5: Verify results ------------------
print(df['Institution Name'])


0       Northwest Institute of Health Sciences
1                                  Saint Louis
2             Illinois Institute Of Technology
3                       Saint Louis University
4                       Saint Louis University
                         ...                  
8553          Lideta Catholic Cathedral School
8554                    Saint Louis University
8555       Tai Solarin University Of Education
8556                    Saint Louis University
8557                    Saint Louis University
Name: Institution Name, Length: 8558, dtype: object


Invalid University Names in Invalid Country: Fixed

In [33]:
# Replace Saint Louis University or St Louis University with Not Applicable if Country is India
df.loc[
    (df["Country"] == "India") & 
    (df["Institution Name"].isin(["Saint Louis University", "St Louis University"])),
    "Institution Name"
] = "Not Applicable"

Additional Corrections of Institution Names

In [34]:
additional_corrections = {
    "Saint Louis": "Saint Louis University",
    "Illinois Institute Of Technology": "Illinois Institute of Technology",
    "Tai Solarin University Of Education": "Tai Solarin University of Education"
    # Add any other variants you find in df['Institution Name'].unique()
}

df['Institution Name'] = df['Institution Name'].replace(additional_corrections)

# Add non-English institutions to your corrections dictionary
institution_corrections.update({
    "ثانوية ابن سينا التأهيلية": "Ibn Sina Secondary School",
    "广州市实验外语学校": "Guangzhou Experimental Foreign Language School",
    "珠海一附国际部": "Zhuhai No.1 Experimental International Division",
    "珠海市一附属实验学校": "Zhuhai No.1 Affiliated Experimental School"
})
df['Institution Name'] = df['Institution Name'].replace(institution_corrections)



Remove extra copyright or unrelated text

In [35]:
import re

def clean_institution_name(name):
    if not isinstance(name, str):
        return None
    # Remove © and anything before the actual institution name
    cleaned = re.sub(r"©.*? ", "", name).strip()
    # Remove multiple spaces
    cleaned = re.sub(r"\s+", " ", cleaned)
    return cleaned if cleaned else None

df['Institution Name'] = df['Institution Name'].apply(clean_institution_name)


Handle unmatched names

In [36]:
df['Institution Name'] = df['Institution Name'].apply(lambda x: x if isinstance(x, str) else "Invalid University")

Verifying by viewing Intitution Names

In [37]:
# Get all unique institution names in the dataset
unique_institutions = df['Institution Name'].unique()

# Sort them alphabetically for easier review
unique_institutions_sorted = sorted(unique_institutions)

# Print the sorted list
for name in unique_institutions_sorted:
    print(name)


2023 Tomtom Openstreetmap Sri Venkateswara College Of Engineering
Aacharya Ng Ranga Agricultural University
Abc Inter College
Abdul Kadir Molla International School
Abdul Raheem
Abdul Wali Khan University
Abdul Wali Khan University Mardan
Abdullah Gul University
Abesan Senior High School
Abia State University
Abia State University Uturu
Abia State University Uturu.
Abn And Prr College Of Science
Abu Dhabi University
Abubakar Tatari Ali Polytechnic Bauchi
Academia-Industry Training (AIT) India program
Academy Of Aerospace And Engineering
Accra Institute Of Technology
Accra Technical University
Ace Engineering College
Acellus Academy
Acharya Nagarjuna University
Achievers University Owo Ondo State
Acp College
Acropolis Institute Of Technology And Research
Adamas University
Adamawa State College Of Education Hong
Addis Ababa Institute Of Technology
Addis Ababa Science And Technology University
Addis Ababa University
Adekunle Ajasin University
Adekunle Ajasin University Akungba Akoko
Adeku

Note: The dataset contained multiple inconsistencies in both the Country and Institution Name columns, including abbreviations, short forms, misspellings, extra text, foreign characters, and invalid entries. To address these issues, a structured cleaning process was implemented. First, country names were standardized using a mapping dictionary to convert abbreviations like US and IND into their full names such as United States and India. Any country not in the valid list was set to None. Next, institution names were standardized in two steps. A comprehensive dictionary of institution corrections replaced abbreviations and short forms with official full names (e.g., Jntuh → Jawaharlal Nehru Technological University Hyderabad). For entries containing additional or unwanted text (e.g., 2023 Tomtom Openstreetmap Sri Venkateswara College Of Engineering) or non-Latin characters (e.g., Chinese or Arabic names), string matching and manual corrections were applied where possible, and unmatched or invalid entries were set to None to clearly indicate missing or unrecognized data. This systematic approach ensured that both country and institution columns became consistent, accurate, and ready for analysis, while preserving the integrity of the dataset by marking ambiguous or invalid entries appropriately.

5. Dealing With Duplicates

Identifying Duplicates

In [38]:
# Check for duplicates across all columns
duplicate_rows = df[df.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

# Check for duplicates based on key identifiers only
duplicate_keys = df[df.duplicated(subset=['Opportunity Id', 'Learner SignUp DateTime'])]
print(f"Number of duplicate key records: {len(duplicate_keys)}")

Number of duplicate rows: 0
Number of duplicate key records: 312


Removing Duplicates

In [39]:
# Remove duplicates based on key identifiers
df = df.drop_duplicates(subset=['Opportunity Id', 'Learner SignUp DateTime'], keep='first')

# Verify
print(f"Remaining records after removing duplicates: {len(df)}")

Remaining records after removing duplicates: 8246


Verifying the duplicate values existing or not

In [40]:
# Check for duplicates across all columns
duplicate_rows = df[df.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

# Check for duplicates based on key identifiers only
duplicate_keys = df[df.duplicated(subset=['Opportunity Id', 'Learner SignUp DateTime'])]
print(f"Number of duplicate key records: {len(duplicate_keys)}")

Number of duplicate rows: 0
Number of duplicate key records: 0


Note : During the data cleaning process, we first checked for exact duplicate rows across all columns and found none, ensuring there were no identical redundancies in the dataset. Next, we examined key-based duplicates using the combination of Opportunity Id and Learner SignUp DateTime, which revealed multiple records sharing the same key identifiers. To maintain data integrity while removing redundancy, we retained only the first occurrence of each duplicate key combination and safely dropped the others. This approach ensured that the dataset remained consistent, free of unnecessary duplicates, and no missing values were introduced, preserving the reliability of the remaining data for analysis.

6. Handling Inconsistent Categorical Data

Inspecting unique values

In [41]:
categorical_cols = ["Opportunity Name", "Opportunity Category", "Gender", "Country",
                    "Institution Name", "Current/Intended Major", "Status Description"]

for col in categorical_cols:
    print(f"Unique values in {col}:")
    print(df[col].unique())
    print("\n")

Unique values in Opportunity Name:
['Career Essentials: Getting Started with Your Professional Journey'
 'Slide Geeks: A Presentation Design Competition' 'Digital Marketing'
 'Health Care Management' 'Innovation & Entrepreneurship'
 'Project Management' 'Data Visualization' 'CPR/AED Certification'
 'Mental and Physical Health Session'
 'Jump Start: Developing your Emotional Intelligence'
 'Join a Student Organisation' 'Upload Your First Year Transcript'
 'Startup Mastery Workshop' 'AI Ethics Challenge'
 'Data Visualization Associate' 'Digital Strategy Virtual Internship'
 'Project Management Associate' 'Business Consulting'
 'UrbanRenew Challenge' 'UX Redesign Challenge'
 'Xperience Design Hackathon' 'Freelance Mastery workshop']


Unique values in Opportunity Category:
['Course' 'Competition' 'Internship' 'Event' 'Engagement']


Unique values in Gender:
['Female' 'Male' "Don'T Want To Specify" 'Other']


Unique values in Country:
['Pakistan' 'India' 'United States' 'United Arab Emirat

Handled Inconsistences

In [43]:
# Safe replacements for Major placeholders
major_mapping_safe = {
    'Oth': 'Other', 'I Am Major': 'Other', 'Otheraassss': 'Other', 'Not Know Ab': 'Other',
    'Non': 'Other', 'Na': 'Other', 'Fresher': 'Other', 'Dropped Out': 'Other',
    'Job': 'Other', 'Student': 'Other', 'Could Computing': 'Other', 'Te': 'Other', 'It': 'Other',
    'Ise': 'Other', 'Pv': 'Other', 'Www': 'Other', 'Shaista': 'Other', 'Ha': 'Other', 'Faizan': 'Other',
    'Zack': 'Other', 'Cybersecurity': 'Cyber Security', 'Data Sceince': 'Data Science',
    'Computer And Infromation Sciences': 'Computer And Information Sciences'
}
df['Current/Intended Major'] = df['Current/Intended Major'].replace(major_mapping_safe)

# Safe replacements for Institution Name
institution_mapping_safe = {
    'Saint Louise University': 'Saint Louis University',
    'Srm University': 'SRM University',
    'Eiilm University': 'EIILM University'  # confirm spelling
    # do NOT replace 'None', keep original to avoid missing
}
df['Institution Name'] = df['Institution Name'].replace(institution_mapping_safe)

# Normalize Gender safely
gender_mapping_safe = {
    "Don'T Want To Specify": 'Prefer Not To Say'
}
df['Gender'] = df['Gender'].replace(gender_mapping_safe)

# Optional: strip spaces and unify capitalization (won't create missing values)
text_cols = ['Opportunity Name', 'Opportunity Category', 'First Name', 
             'Gender', 'Country', 'Institution Name', 'Current/Intended Major', 
             'Status Description']

for col in text_cols:
    df[col] = df[col].astype(str).str.strip().str.title()

# Check cleaned values
for col in ['Current/Intended Major', 'Institution Name', 'Gender']:
    print(f"{col} unique values after safe cleaning:")
    print(df[col].unique())
    print('-'*50)


Current/Intended Major unique values after safe cleaning:
['Radiology' 'Information Systems' 'Computer Science'
 'Mechanical Engineering' 'Computer Science And Engineering'
 'Artificial Intelligence' 'Robotics And Automation Engineering'
 'Data Visualization' 'Business Administration' 'Public Health'
 'Architecture' 'Computer Science And Information Systems' 'Biology'
 'Economics' 'Other' 'Mathematics' 'Bioinformatics'
 'Biomedical Engineering' 'Electrical And Electronic Engineering'
 'Business And Management Studies' 'Electrical And Computer Engineering'
 'Accounting And Finance' 'Secretarial' 'Data Science' 'Statistics'
 'Electronics And Communication' 'Computer Information Systems'
 'Management Information Systems' 'Project Management' 'Medicine'
 'Information' 'Information Technology' 'Actuarial Mathematics'
 'Software Engineering' 'Biological Sciences'
 'Urban And Housing Development' 'Human Resources' 'Cyber Security'
 'Data Analytics' 'Computer Engineering' 'Environmental Scienc

Note: In our dataset, several categorical fields contained inconsistencies, including placeholder values like "Oth," "I Am Major," or variations in institution names and gender entries. To ensure data quality without introducing missing values, we applied a careful normalization process. For the Current/Intended Major field, all ambiguous or placeholder values were safely mapped to "Other," while for Institution Name, inconsistent abbreviations and misspellings were corrected to their full official names or marked as "Invalid University" where appropriate. Similarly, the Gender field was standardized by mapping entries like "Don'T Want To Specify" to "Prefer Not To Say." Beyond these mappings, all categorical text columns were cleaned by stripping leading/trailing spaces and converting entries to title case, ensuring uniformity and consistency across the dataset. This approach preserved the integrity of the original data while eliminating ambiguity, making it reliable for further analysis and modeling.

Exporting Cleaned Dataset After Data_Cleaning Process

In [44]:
# Export the cleaned dataset to CSV
df.to_csv("Cleaned_Preprocessed_Dataset.csv", index=False)

# Finally The Dataset is Cleaned ! 

Total Rows and columns in Cleaned Dataset

In [45]:
df_cleaned = pd.read_csv("Cleaned_Preprocessed_Dataset.csv")

# Total rows and columns
total_rows, total_cols = df_cleaned.shape
print(f"Total Rows: {total_rows}")
print(f"Total Columns: {total_cols}")

Total Rows: 8246
Total Columns: 16


Viewing first few rows of the cleaned Dataset

In [46]:
from IPython.display import display
display(df.head(5))

,Learner SignUp DateTime,Opportunity Id,Opportunity Name,Opportunity Category,Opportunity End Date,First Name,Date of Birth,Gender,Country,Institution Name,Current/Intended Major,Entry created at,Status Description,Status Code,Apply Date,Opportunity Start Date
0,06/14/2023 12:30:35,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started With Your P...,Course,06/29/2024 18:52:39,Faria,01/12/2001,Female,Pakistan,Northwest Institute Of Health Sciences,Radiology,03/11/2024 12:01:41,Started,1080,06/14/2023 12:36:09 PM,11/03/2022 18:30:39
1,05/01/2023 05:29:16,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started With Your P...,Course,06/29/2024 18:52:39,Poojitha,08/16/2000,Female,India,Saint Louis University,Information Systems,03/11/2024 12:01:41,Started,1080,05/01/2023 06:08:21 AM,11/03/2022 18:30:39
2,04/09/2023 20:35:08,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started With Your P...,Course,06/29/2024 18:52:39,Emmanuel,01/27/2002,Male,United States,Illinois Institute Of Technology,Computer Science,03/11/2024 12:01:41,Started,1080,01/01/1900 12:00:00 AM,11/03/2022 18:30:39
3,08/29/2023 05:20:03,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started With Your P...,Course,06/29/2024 18:52:39,Amrutha Varshini,11/01/1999,Female,United States,Saint Louis University,Information Systems,03/11/2024 12:01:41,Team Allocated,1070,10/09/2023 10:02:42 PM,11/03/2022 18:30:39
4,01/06/2023 15:26:36,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started With Your P...,Course,06/29/2024 18:52:39,Vinay Varshith,04/19/2000,Male,United States,Saint Louis University,Computer Science,03/11/2024 12:01:41,Started,1080,01/06/2023 03:40:10 PM,11/03/2022 18:30:39
